In [1]:
from bs4 import *
import requests

In [2]:
CNN_NEWS = "https://edition.cnn.com/world" # this url directs to the world news section 
MAINPAGE = "https://edition.cnn.com"

In [3]:
def get_soup(url : str)-> BeautifulSoup:
    article = requests.get(url).text
    return BeautifulSoup(article)

def get_content(article_soup : BeautifulSoup):
    paragraphs = [paragraph.text for paragraph in article_soup.find_all(is_paragraph)]
    string = ""
    for paragraph in paragraphs:
        string = f"{string} {paragraph}"
    return string

'''def get_news(article_url : str) -> str:
    article_soup = get_soup(article_url)
    return get_content(article_soup)'''

def is_article(tag) -> bool:
    return tag.has_attr("data-open-link") and tag.has_attr("data-word-count")


def get_article_links(article_soup : BeautifulSoup):
    results = article_soup.find_all(is_article)
    results_list = list(results)
    results_list = [f"{MAINPAGE}{result.attrs['data-open-link']}" for result in results]
    '''for result in results:
        #print(dir(result))
        print(f"{MAINPAGE}{result.attrs['data-open-link']}")'''
    return results_list

def get_headline(article_soup : BeautifulSoup):
    return article_soup.find("h1").text

def is_paragraph(tag) -> bool:
    return tag.has_attr("data-component-name") and tag.name == "p"

def is_author(tag) -> bool:
    return tag.get_attribute_list("class")[0] == "byline__name"

def get_authors(article_soup : BeautifulSoup):
    author_tags = article_soup.find_all(is_author)
    names = [tag.string for tag in author_tags]
    return names

In [4]:
# This code cell lists out article links from the main page
mainpage_soup = get_soup(MAINPAGE)
articles = get_article_links(mainpage_soup)
print(articles)

['https://edition.cnn.com/2023/10/23/middleeast/israel-gaza-hamas-war-monday-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/23/politics/tom-emmer-2020-election-kfile/index.html', 'https://edition.cnn.com/2023/10/23/world/iran-armita-geravand-brain-dead-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/22/world/mountain-chicken-frog-endangered-chytrid-disease-scn/index.html', 'https://edition.cnn.com/travel/fukuoka-japan-street-food-yatai-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/23/world/apollo-17-moon-age-crystals-scn/index.html', 'https://edition.cnn.com/2023/10/23/entertainment/dwayne-johnson-wax-figure/index.html', 'https://edition.cnn.com/2023/10/24/middleeast/israel-hamas-gaza-war-tuesday-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/24/middleeast/israel-hostages-freed-lifshitz-cooper-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/23/middleeast/gaza-hospitals-doctors-disaster-intl-hnk/index.html', 'https://edition.cnn.com/2023/10/23/poli

In [8]:
# This code cell returns the news paragraphs inside a specific article. By changing the link to another
# CNN article, the whole process should still work.
ARTICLE = articles[1]
article_soup = get_soup(ARTICLE)
get_content(article_soup)

' \n      In the aftermath of the 2020 election, Tom Emmer, a leading Republican candidate to be speaker of the House, baselessly said there were “questionable” practices in the 2020 presidential election. \n   \n      Later, Emmer signed an amicus brief in support of a last-ditch Texas lawsuit seeking to throw out the results in key swing states.  \n   \n      Though he would vote to certify the results on January 6, 2021, the comments and actions show Emmer flirted with some of the same election denial rhetoric as far-right members of the Republican caucus. \n   \n      Speaking with the radio show for the far-right publication Breitbart News 12 days after the election, Emmer baselessly suggested that mail-in ballots might have “skewed” the election against Trump. \n   \n      “I think that you will see the courts, if nothing else, this president is making sure that he stays focused and his team stays focused on these questionable election practices,” Emmer said. “We’re gonna find ou

In [6]:
# Below, the authors of a given article are given.
get_authors(article_soup)

['Helen Regan',
 'Kareem Khadder',
 'Eyad Kourdi',
 'Abeer Salman',
 'Lauren Said-Moorhouse',
 'Tara John']